# Import

In [1]:
import numpy as np
import pandas as pd
import os
import tqdm
import torch
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import roc_auc_score, mean_squared_error
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import sampler
from time import time
import numpy as np

In [3]:
us = os.getcwd()
print(us)

if 'liliyaivannikova' in us:
    PATH = r'/Users/liliyaivannikova/Documents/project/ml-latest/'
    movies = pd.read_csv(PATH + r'movies.csv')
    rating = pd.read_csv(PATH + r'ratings.csv')
elif 'Владислав' in us:
    movies = pd.read_csv(r'dataset/movies.csv')
    rating = pd.read_csv(r'dataset/ratings.csv')
    
print(f'''
movies: {movies.shape}
rating: {rating.shape}
''')

/Users/liliyaivannikova/Documents/project/git/RecSys_Films/recsyc_part1

movies: (86537, 3)
rating: (33832162, 4)



In [4]:
movies.columns = [col.upper() for col in movies.columns]
movies['REALEASE'] = movies['TITLE'].str.extract("\((\d{4})\)", expand=True)
movies['REALEASE'] = pd.to_datetime(movies['REALEASE'], format='%Y')
movies['REALEASE'] = movies['REALEASE'].dt.year
movies['TITLE'] = movies['TITLE'].str[:-7]

# dummy-кодирование жанров
dfx = movies['GENRES'].str.get_dummies(sep='|')
for col in dfx.columns:
    dfx[col] = dfx[col].astype('int8')
    
movies = pd.concat([movies, dfx], axis=1) #.drop(columns=['GENRES'])
movies['TITLE'] = movies['TITLE'].astype('category')
movies['REALEASE'] = movies['REALEASE'].astype('float16')
movies['GENRES'] = movies['GENRES'].str.replace('|', ',')

rating.columns = [col.upper() for col in rating.columns]
rating['TIMESTAMP'] = pd.to_datetime(rating['TIMESTAMP'], unit='s')

/var/folders/pf/9t70d84n0wbgzdftsf9jkvtw0000gn/T/ipykernel_6504/1871982909.py:15: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['GENRES'] = movies['GENRES'].str.replace('|', ',')


# Фильтрация строк

In [5]:
# считаем кол-во оценок в рамках userid
rating_stat = rating.groupby('USERID')['MOVIEID'].count().reset_index()
rating_stat['MOVIEID'].quantile(0.4)

21.0

In [6]:
#rating_stat['FLAG_05'] = np.where(rating_stat['MOVIEID'] > rating_stat['MOVIEID'].quantile(0.05), 1, 0)
'''при текущих симуляцих мы отфильтровали наш набор данных, оставив для построения только userid, которые
оценили больше 20 фильмов
'''
rating_stat['FLAG_05'] = np.where(rating_stat['MOVIEID'] >= rating_stat['MOVIEID'].quantile(0.4), 1, 0)
rating = rating.merge(rating_stat[['USERID', 'FLAG_05']], how = 'left', on = 'USERID')
rating.head()

,USERID,MOVIEID,RATING,TIMESTAMP,FLAG_05
0,1,1,4.0,2008-11-03 17:52:19,1
1,1,110,4.0,2008-11-05 06:04:46,1
2,1,158,4.0,2008-11-03 17:31:43,1
3,1,260,4.5,2008-11-03 18:00:04,1
4,1,356,5.0,2008-11-03 17:58:39,1


In [7]:
'''
По результатам EDA
'''
# удаляем пропуски
rating = rating[~((rating.TIMESTAMP.isna())|(rating.USERID.isna())|(rating.RATING.isna()))]
# удаляем странного юзера 
rating.query('USERID != 189614', inplace = True)
# удалим юзеров, у которых оценок меньше 3 (по 5 квантилю)
rating.query('FLAG_05 == 1', inplace = True)
print(rating.shape)

(32346391, 5)


# train-test-split

In [8]:
rating.rename({'USERID':'user_id',
            'MOVIEID':'item_id',
            'RATING':'weight',
            'TIMESTAMP':'datetime'}, axis=1, inplace=True)

movies.rename({'MOVIEID':'item_id'}, axis=1, inplace=True)

In [277]:

class CriteoDataset(Dataset):
     
    def __init__(self, data, train=True):
        
        self.train = train

        if not self._check_exists:
            raise RuntimeError('Dataset not found.')

        if self.train:
            self.train_data = data.iloc[:, :-1].values
            self.target = data.iloc[:, -1].values
        else:
            self.test_data = data.iloc[:, :-1].values
    
    def __getitem__(self, idx):
        if self.train:
            dataI, targetI = self.train_data[idx, :], self.target[idx]
            Xi = torch.from_numpy(dataI.astype(np.int32)).unsqueeze(-1)
            Xv = torch.from_numpy(np.ones_like(dataI))
            return Xi, Xv, targetI
        else:
            dataI = self.test_data.iloc[idx, :]
            Xi = torch.from_numpy(dataI.astype(np.int32)).unsqueeze(-1)
            Xv = torch.from_numpy(np.ones_like(dataI))
            return Xi, Xv

    def __len__(self):
        if self.train:
            return len(self.train_data)
        else:
            return len(self.test_data)

    def _check_exists(self):
        return os.path.exists(self.root)
    
    
class DeepFM(nn.Module):

    def __init__(self, feature_sizes, embedding_size=4,
                 hidden_dims=[32, 32], num_classes=1, dropout=[0.5, 0.5], 
                 use_cuda=True, verbose=False):
        """
        Initialize a new network

        Inputs: 
        - feature_size: A list of integer giving the size of features for each field.
        - embedding_size: An integer giving size of feature embedding.
        - hidden_dims: A list of integer giving the size of each hidden layer.
        - num_classes: An integer giving the number of classes to predict. For example,
                    someone may rate 1,2,3,4 or 5 stars to a film.
        - batch_size: An integer giving size of instances used in each interation.
        - use_cuda: Bool, Using cuda or not
        - verbose: Bool
        """
        super().__init__()
        self.field_size = len(feature_sizes)
        self.feature_sizes = feature_sizes
        self.embedding_size = embedding_size
        self.hidden_dims = hidden_dims
        self.num_classes = num_classes
        self.dtype = torch.long
        self.bias = torch.nn.Parameter(torch.randn(1))
        self.num_correct = 0
        self.num_samples = 0
        """
            check if use cuda
        """
        if use_cuda and torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')
        """
            init fm part
        """
        self.fm_first_order_embeddings = nn.ModuleList(
            [nn.Embedding(feature_size, 1) for feature_size in self.feature_sizes])
        self.fm_second_order_embeddings = nn.ModuleList(
            [nn.Embedding(feature_size, self.embedding_size) for feature_size in self.feature_sizes])
        """
            init deep part
        """
        all_dims = [self.field_size * self.embedding_size] + \
            self.hidden_dims + [self.num_classes]
        for i in range(1, len(hidden_dims) + 1):
            setattr(self, 'linear_'+str(i),
                    nn.Linear(all_dims[i-1], all_dims[i]))
            # nn.init.kaiming_normal_(self.fc1.weight)
            setattr(self, 'batchNorm_' + str(i),
                    nn.BatchNorm1d(all_dims[i]))
            setattr(self, 'dropout_'+str(i),
                    nn.Dropout(dropout[i-1]))

    def forward(self, Xi, Xv):
        """
        Inputs:
        - Xi: A tensor of input's index, shape of (N, field_size, 1)
        - Xv: A tensor of input's value, shape of (N, field_size, 1)
        """
        
        fm_first_order_emb_arr = [(torch.sum(emb(Xi[:, i, :]), 1).t() * Xv[:, i]).t() for i, emb in enumerate(self.fm_first_order_embeddings)]
        fm_first_order = torch.cat(fm_first_order_emb_arr, 1)
        fm_second_order_emb_arr = [(torch.sum(emb(Xi[:, i, :]), 1).t() * Xv[:, i]).t() for i, emb in enumerate(self.fm_second_order_embeddings)]
        fm_sum_second_order_emb = sum(fm_second_order_emb_arr)
        fm_sum_second_order_emb_square = fm_sum_second_order_emb * \
            fm_sum_second_order_emb  # (x+y)^2
        fm_second_order_emb_square = [
            item*item for item in fm_second_order_emb_arr]
        fm_second_order_emb_square_sum = sum(
            fm_second_order_emb_square)  # x^2+y^2
        fm_second_order = (fm_sum_second_order_emb_square -
                           fm_second_order_emb_square_sum) * 0.5
        """
            deep part
        """
        deep_emb = torch.cat(fm_second_order_emb_arr, 1)
        deep_out = deep_emb
        for i in range(1, len(self.hidden_dims) + 1):
            deep_out = getattr(self, 'linear_' + str(i))(deep_out)
            deep_out = getattr(self, 'batchNorm_' + str(i))(deep_out)
            deep_out = getattr(self, 'dropout_' + str(i))(deep_out)
        """
            sum
        """
        total_sum = torch.sum(fm_first_order, 1) + \
                    torch.sum(fm_second_order, 1) + torch.sum(deep_out, 1) + self.bias
        return total_sum

    def fit(self, loader_train, loader_val, optimizer, epochs=100, verbose=False, print_every=100):

        model = self.train().to(device=self.device)
        criterion = F.binary_cross_entropy_with_logits
        
        train_losses, val_losses = [], []

        for ii in range(epochs):
            
            print('Epoch: ', ii, '\n')
            print('train start \n')
            
            running_loss = 0
            
            for t, (xi, xv, y) in enumerate(loader_train):
                xi = xi.to(device=self.device, dtype=self.dtype)
                xv = xv.to(device=self.device, dtype=torch.float)
                y = y.to(device=self.device, dtype=torch.float)
                
                total = model(xi, xv)
                loss = criterion(total, y)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                with torch.no_grad():
                    running_loss += loss.item() * xi.shape[0]
                
                if verbose and t % print_every == 0:
                    print('Iteration %d, train loss = %.4f' % (t, loss.item()))
                    print()
                    
                if verbose and t % print_every*5 == 0:    
                    train_losses += [running_loss / len(loader_train.dataset)]
                    
            model.eval()
            running_loss = 0  
                
            with torch.no_grad():
                print('test start \n')
                if ii == epochs-1:
                    pred_fin = []
                for t, (xi, xv, y) in enumerate(loader_val):
                    xi = xi.to(device=self.device, dtype=self.dtype)
                    xv = xv.to(device=self.device, dtype=torch.float)
                    y = y.to(device=self.device, dtype=torch.float)

                    total = model(xi, xv)
                    loss = criterion(total, y)
                    running_loss += loss.item() * xi.shape[0]

                    if ii == epochs-1:
                        pred_fin.extend(total)
                        
                    if verbose and t % print_every == 0:
                        print('Iteration %d, test loss = %.4f' % (t, loss.item()))
                        print()
                    
                    if verbose and t % print_every == 0:    
                        val_losses += [running_loss / len(loader_val.dataset)]
            
                if ii == epochs-1:
                    return np.array(pred_fin), train_losses, val_losses

In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [129]:
'''
бьем на train/test по времени (20 последних оценок идут в test)
'''
rating.sort_values(['user_id', 'datetime'], inplace=True)

train_ratings, test_ratings = [], []
num_test_samples = 10

for userId, user_data in rating.groupby('user_id'):
    train_ratings += [user_data[:-num_test_samples]]
    test_ratings += [user_data[-num_test_samples:]]

train_ratings = pd.concat(train_ratings)
test_ratings = pd.concat(test_ratings)
train_ratings.shape, test_ratings.shape

((30357121, 5), (1989270, 5))

In [142]:
test_target = test_ratings['weight'].copy()
test_ratings.drop(['datetime', 'FLAG_05'], axis=1, inplace=True)
train_target = train_ratings['weight'].copy()
train_ratings.drop(['datetime', 'FLAG_05'], axis=1, inplace=True)

In [200]:
train_ratings['weight'] = np.where(train_ratings['weight']>3, 1, 0)
test_ratings['weight'] = np.where(test_ratings['weight']>3, 1, 0)

In [202]:
train_data = CriteoDataset(train_ratings, train=True)
loader_train = DataLoader(train_data, batch_size=1028*8)
val_data = CriteoDataset(test_ratings, train=True)
loader_val = DataLoader(val_data, batch_size=1028*8)

# deepFM

In [279]:
%time
feature_sizes = [500000, 500000]

model = DeepFM(feature_sizes, embedding_size=64,
                 hidden_dims=[64, 64], num_classes=2)
optimizer = optim.Adam(model.parameters(), lr=1e-6, weight_decay=0.0)
preds, train_losses, val_losses = model.fit(loader_train, loader_val, optimizer, epochs=5, verbose=True)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.91 µs
Epoch:  0 

train start 

Iteration 0, train loss = 5.7882

Iteration 100, train loss = 5.5469

Iteration 200, train loss = 5.3130

Iteration 300, train loss = 5.1381

Iteration 400, train loss = 5.5301

Iteration 500, train loss = 5.6922

Iteration 600, train loss = 5.1699

Iteration 700, train loss = 5.3609

Iteration 800, train loss = 5.5529

Iteration 900, train loss = 5.1259

Iteration 1000, train loss = 5.3983

Iteration 1100, train loss = 5.3577

Iteration 1200, train loss = 5.1770

Iteration 1300, train loss = 5.1913

Iteration 1400, train loss = 5.1780

Iteration 1500, train loss = 5.4480

Iteration 1600, train loss = 5.0968

Iteration 1700, train loss = 5.3514

Iteration 1800, train loss = 5.7184

Iteration 1900, train loss = 5.6185

Iteration 2000, train loss = 5.0941

Iteration 2100, train loss = 5.3604

Iteration 2200, train loss = 5.3603

Iteration 2300, train loss = 5.0595

Iteration 2400, train loss = 5.073

In [286]:
test_ratings['PRED'] = preds.copy()

In [305]:
test_ratings['PD'] = F.sigmoid(torch.from_numpy(test_ratings['PRED'].values).unsqueeze(-1))

In [288]:
# test_ratings['TARGET'] = np.where(test_ratings['weight']>3, 1, 0) уже сделано

In [289]:
test_ratings.head()

,user_id,item_id,weight,PRED,TARGET
33,1,4246,1,8.608948,0
53,1,8969,1,1.404488,0
35,1,4699,1,8.815282,0
46,1,7162,1,11.864023,0
54,1,8981,1,0.792975,0


In [306]:
test_ratings.sort_values(['user_id', 'PD'], ascending=[True, False], inplace=True)

In [307]:
test_ratings.head()

,user_id,item_id,weight,PRED,TARGET,PD
27,1,2671,1,20.422924,0,1.000000
1,1,110,1,15.521448,0,1.000000
46,1,7162,1,11.864023,0,0.999993
35,1,4699,1,8.815282,0,0.999852
33,1,4246,1,8.608948,0,0.999818


In [312]:
from tqdm import tqdm
temp_t = []

for user in tqdm(test_ratings['user_id'].unique()):
    temp_t.append(test_ratings.loc[test_ratings['user_id'] == user, 'weight'][:5].sum())

100%|██████████| 198927/198927 [02:46<00:00, 1197.54it/s]


In [314]:
sum(temp_t) / len(temp_t)

3.41908840931598

In [315]:
from tqdm import tqdm
temp_t = []

for user in tqdm(test_ratings['user_id'].unique()):
    temp_t.append(test_ratings.loc[test_ratings['user_id'] == user, 'weight'][:10].sum())

100%|██████████| 198927/198927 [02:42<00:00, 1221.11it/s]


In [316]:
sum(temp_t) / len(temp_t)

6.822236297737361

In [317]:
from tqdm import tqdm
temp_t = []

for user in tqdm(test_ratings['user_id'].unique()):
    temp_t.append(test_ratings.loc[test_ratings['user_id'] == user, 'weight'][:3].sum())

100%|██████████| 198927/198927 [02:46<00:00, 1197.03it/s]


In [318]:
sum(temp_t) / len(temp_t)

2.0520743790435687